In [5]:
import pandas as pd
import numpy as np



In [7]:
df = pd.read_csv('AlgoMatch_ Exploring the Student Social Game on Bumble (Responses) - Form Responses 1.csv')


In [9]:
# parse the A1, A2 etc. on each column name and set as new column names
df.columns = df.columns.str.extract(r'([A-Z]\d+)')[0]


In [13]:
# drop unnecessary columns (C1 to C15)
df = df.drop(columns=[col for col in df.columns if isinstance(col, str) and col.startswith('C')])


In [16]:
# rename nan column to 'dateTime' 
df = df.rename(columns={np.nan: 'dateTime'})
# convert dateTime to datetime format
df['dateTime'] = pd.to_datetime(df['dateTime'], errors='coerce')

In [18]:
df.columns

Index(['dateTime', 'A1', 'A2', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9',
       'A10', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10',
       'D11', 'D12', 'D13', 'D14', 'D15', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6',
       'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15'],
      dtype='object', name=0)

In [28]:
# create new data frame for columns A1 to A10 and D1 to D15, filter out the resulting rows with Nan from columns starting with D
df_A_D = df.loc[:, df.columns.str.startswith(('A', 'D'))]
df_A_D = df_A_D.dropna(subset=[col for col in df_A_D.columns if col.startswith('D')])
# create new data frame for columns A1 to A10 and B1 to B15,  filter out the resulting rows with Nan from columns starting with B
df_A_B = df.loc[:, df.columns.str.startswith(('A', 'B'))]
df_A_B = df_A_B.dropna(subset=[col for col in df_A_B.columns if col.startswith('B')])

#print out side by side column names of df_A_D and df_A_B in zip
for col_A_D, col_A_B in zip(df_A_D.columns, df_A_B.columns):
    print(f"{col_A_D:>5} | {col_A_B:>5}")


   A1 |    A1
   A2 |    A2
   A2 |    A2
   A3 |    A3
   A4 |    A4
   A5 |    A5
   A6 |    A6
   A7 |    A7
   A8 |    A8
   A9 |    A9
  A10 |   A10
   D1 |    B1
   D2 |    B2
   D3 |    B3
   D4 |    B4
   D5 |    B5
   D6 |    B6
   D7 |    B7
   D8 |    B8
   D9 |    B9
  D10 |   B10
  D11 |   B11
  D12 |   B12
  D13 |   B13
  D14 |   B14
  D15 |   B15


### Analysis of Bumble Date DataFrame 

### Analysis of Bumble BFF DataFrame